# Load and clean the data

* Load the metadata prepared in `../00_load_metadata.ipynb`
* Keep only papers on natural language processing: original category 'cs.CL' (Computation and Language)
* Load abstracts prepared in `../00_load_abstracts.ipynb`, merge with metadata dataframe
* Check that all entries have an abstract
* Keep only research papers (research papers are papers that are not review papers).

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split


In [2]:
%%time

# Load the metadata downloaded from archive
arxiv_metadata = pd.read_csv('../data/arxiv_metadata.csv.zip', index_col=0)

<timed exec>:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 8.99 s, sys: 1.11 s, total: 10.1 s
Wall time: 10.1 s


In [3]:
# Keep only papers on natural language processing: original category 'cs.CL' (Computation and Language)
nlp_idx = ['cs.CL' in subject for subject in arxiv_metadata['categories']]
arxiv_nlp = arxiv_metadata[nlp_idx]

In [4]:
%%time

# load abstracts extracted data in notebook 00_load_abstracts
arxiv_abstracts = pd.read_csv('../data/arxiv_abstracts.csv.zip', index_col=0)

CPU times: user 9.62 s, sys: 1.14 s, total: 10.8 s
Wall time: 12.1 s


<timed exec>:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.


In [5]:
# merge with metadata dataframe
arxiv_abstracts_nlp = arxiv_abstracts[arxiv_abstracts.id.isin(arxiv_nlp.id)]
arxiv_nlp_merged = pd.merge(arxiv_nlp, arxiv_abstracts_nlp, on='id')

In [6]:
# check that all entries have an abstract
idx = arxiv_nlp_merged['abstract'].isna()
arxiv_nlp_merged = arxiv_nlp_merged[~idx]

In [7]:
# Keep only research papers (research papers are papers that are not review papers).
research_paper_idx = ['systematic literature review' not in abstract.lower() for abstract in arxiv_nlp_merged.abstract]
arxiv_nlp_merged = arxiv_nlp_merged[research_paper_idx]

In [8]:
print(f"There are {len(arxiv_nlp_merged)} research papers on NLP with an abstract in the dataset.")

There are 55943 research papers on NLP with an abstract in the dataset.


## Save

In [11]:
import zipfile as zf

with zf.ZipFile('../data/arxiv_nlp.csv.zip', 'w') as ziparchive:
    ziparchive.writestr('arxiv_nlp.csv', arxiv_nlp_merged.to_csv())
